##  Kernels

But this time we refine the mesh around the surface, base level and the layer where the buoyancy is located.

The mesh is adjusted each time we want to compute a new point. This means we can compute as many vertical points
as we want ... 



In [ ]:
%matplotlib inline
import matplotlib.pyplot as pyplot

import underworld as uw
import math
from underworld import function as fn
import glucifer.pylab as plt

In [ ]:
dim = 2
meshX = 128
meshY = 128

Q1dP0Mesh = uw.mesh.FeMesh_Cartesian( elementType='Q1/dQ0',
                                      elementRes=(meshX,meshY), 
                                      minCoord=(0.,0.), maxCoord=(1.,1.)  )

Q1Mesh =  Q1dP0Mesh # returns outer mesh by default
dP0Mesh = Q1dP0Mesh.subMesh

velocityMesh =  Q1Mesh
pressureMesh = dP0Mesh

xCoordFn = fn.input()[0]
yCoordFn = fn.input()[1]


In [ ]:
# create & zero meshvariables
velocityField    = uw.mesh.MeshVariable( mesh=velocityMesh,   nodeDofCount=dim )
pressureField    = uw.mesh.MeshVariable( mesh=pressureMesh,   nodeDofCount=1 )
temperatureField = uw.mesh.MeshVariable( mesh=velocityMesh,   nodeDofCount=1 )

velocityField.data[:] = [0.,0.]
pressureField.data[:] = 0.
temperatureField.data[:] = 0.

In [ ]:
IWalls = velocityMesh.specialSets["MinI_VertexSet"] + velocityMesh.specialSets["MaxI_VertexSet"]
JWalls = velocityMesh.specialSets["MinJ_VertexSet"] + velocityMesh.specialSets["MaxJ_VertexSet"]

freeslipBC = uw.conditions.DirichletCondition(    variable=velocityField, 
                                                  indexSetsPerDof=(IWalls,JWalls) )

# temperature does not evolve with time so no bcs are needed

In [ ]:
# UW function to define a layered viscosity

layer1Viscosity = fn.misc.constant(1.0)
layer2Viscosity = fn.misc.constant(1.0)
layer3Viscosity = fn.misc.constant(1.0)

layer1Depth = 0.9
layer2Depth = 0.7
layer3Depth = 0.0

xCoordFn = fn.input()[0]
yCoordFn = fn.input()[1]

fn_viscosity = fn.branching.conditional(
    [ ( yCoordFn > layer1Depth , layer1Viscosity ),
      ( yCoordFn > layer2Depth , layer2Viscosity ), 
      (                   True , layer3Viscosity )  ] )

# UW function to compute the total stress and vertical stress at any given point
# And a filter function to grab the coordinates at the node points

stresstensorFn =  2.* fn_viscosity*fn.tensor.symmetric( velocityField.fn_gradient ) - pressureField
verticalStressFn = stresstensorFn[1]

In [ ]:
velocityField.fn_gradient

In [ ]:
# Setup a stokes system
# For PIC style integration, we include a swarm for the a PIC integration swarm is generated within.
# For gauss integration, simple do not include the swarm. Nearest neighbour is used where required.

forceFn = (0.0, 1.0) * temperatureField

stokesEqn = uw.systems.Stokes(velocityField=velocityField, 
                              pressureField=pressureField,
                              conditions=[freeslipBC,],
                              fn_viscosity=fn.exception.SafeMaths(fn_viscosity), 
                              fn_bodyforce=forceFn )

solver=uw.systems.Solver(stokesEqn)

solver.options.main.Q22_pc_type='uwscale'  # also try 'gtkg', 'gkgdiag' and 'uwscale'
solver.options.main.penalty = 1.0
solver.options.A11.ksp_rtol=1e-6
solver.options.scr.ksp_rtol=1e-5
solver.options.scr.use_previous_guess = True
solver.options.scr.ksp_set_min_it_converge = 1
solver.options.scr.ksp_set_max_it = 100
solver.options.mg.levels = 5
solver.options.mg.mg_levels_ksp_type = 'chebyshev'
solver.options.mg_accel.mg_accelerating_smoothing = True
solver.options.mg_accel.mg_accelerating_smoothing_view = False
solver.options.mg_accel.mg_smooths_to_start = 1

# Load this solver onto the equation system (displacing the default)
stokesEqn.solve = solver.solve

### Node refinement

Here is a simple way to map mesh points to match a concentration function which is supplied (on the unit interval). These functions assume the mesh size variables above so take care calling them if you redefine the mesh parameters later (e.g. the max / min coords and the resolution).

In [ ]:
import numpy as np
import math

def gauss(r,a):
   return np.exp(-r**2/a**2) 

def three_way_node_density(alpha, beta, yLoc):
    """
    This is a function to return a new distribution of point densities on the unit interval
    in three gaussian-shaped regions (surface, base and one specified - 
    presumed to be where the forcing function is concentrated)
    
    """

    count = len(yLoc)
    new_vpoints            = np.zeros(count)

    refine1 = gauss(yLoc, beta) 
    refine2 = gauss(yLoc-1.0, beta)
    refine3 = gauss(yLoc-alpha, beta)

    node_density_function = (refine1 + refine2 + refine3)
                
    return node_density_function


def unit_interval_node_spacer(node_count, separation, yd):

    from scipy import interpolate as interpolate
    from scipy.misc import derivative as derivative

    count = node_count

    ys = np.linspace(0.0,1.0,count)
    
    # separation = 10.0 * np.sin(4.0*math.pi*yd)**2
    separation -= separation.min() - 1.0
    separation_fn_rbf = interpolate.Rbf(yd, separation)
    separation_fn_rbf_deriv = derivative(separation_fn_rbf, ys, dx=0.5/count)

    sep = ys.copy() 
    sep[1:-1] = 0.5 * (ys[2:] - ys[0:-2])
    sep[-1] = sep[-2]
    sep[0] = sep[1]

    sep_ratio = sep / sep.min() 

    node_separation_anomaly = (sep_ratio - separation_fn_rbf(ys))
    node_separation_anomaly_rbf = interpolate.Rbf(ys, node_separation_anomaly, smooth=0.5)
    node_flux =  derivative(node_separation_anomaly_rbf, ys, dx=0.5/count)

    node_separation_anomaly0 = node_separation_anomaly.copy()

    delta = np.abs(sep / node_flux).min() / count
    residual = (node_separation_anomaly**2).sum()
    residual0 = residual
    loops = 0
    while loops < 1000 and residual > residual0 * 0.001:

        y_old = ys.copy()

        for i in range(1, count-1):
            ys += node_flux * delta  # ? SOR

        ys /= (ys.max() - ys.min()) 
        ys -= ys.min()

        ys = 0.75 * ys + 0.25 * y_old

        sep = ys.copy() 
        sep[1:-1] = 0.5 * (ys[2:] - ys[0:-2])
        sep[-1] = sep[-2]
        sep[0] = sep[1]

        sep_ratio = sep / sep.min() 

        node_separation_anomaly = (sep_ratio - separation_fn_rbf(ys))
        node_separation_anomaly_rbf = interpolate.Rbf(ys, node_separation_anomaly, smooth=0.5)
        node_flux = derivative(node_separation_anomaly_rbf, ys, dx=1.0/count)

        residual = (node_separation_anomaly**2).sum()

        delta = np.abs(sep / node_flux).min() / count
        loops += 1

#    print loops, "iterations, residual = ", residual / residual0
    
    return ys, sep_ratio


# Need to fix this for non-unit mesh sizes #

def remap_vmesh_pmesh_y(alpha, beta, intensity):
    """
    remap the mesh in x to give better resolution on boundaries and around one horizontal line
    """
    with velocityMesh.deform_mesh():
        yd = np.linspace(0,1.0,1000)
        separation = intensity * ( 1.0 - three_way_node_density(alpha, beta, yd)) 

        new_yVcoords, sep = unit_interval_node_spacer(meshY+1, separation, yd)

        vpoints = velocityMesh.data.reshape(meshY+1,meshX+1,2)

        meshMinY = velocityMesh.minCoord[1]
        meshDY =   velocityMesh.maxCoord[1] - velocityMesh.minCoord[1]

        for column in range(0,meshX+1):
            vpoints[:,column,1] = new_yVcoords[:] * meshDY + meshMinY

        # This will have changed the mesh since {v,p}points is a view into the original mesh data    

        actual_level = 0
        actual_coord = 0.0
        distance = 1.0

        for i in range(0,meshY+1):
            this_distance = np.abs(new_yVcoords[i] - alpha)
            if this_distance < distance:
                distance = this_distance
                actual_coord = new_yVcoords[i]
                actual_level = i

        return actual_coord, actual_level


def remap_vmesh_pmesh_x(alpha, beta, intensity):
    """
    remap the mesh in x to give better resolution on boundaries and around one vertical line.
    Caution, the kernel routines / ffts currently assume even spacing laterally !!
    """
    with velocityMesh.deform_mesh():

        xd = np.linspace(0,1.0,1000)
        separation = intensity * ( 1.0 - three_way_node_density(alpha, beta, xd)) 

        new_xVcoords, sep = unit_interval_node_spacer(meshX+1, separation, xd)

        vpoints = velocityMesh.data.reshape(meshY+1,meshX+1,2)

        meshMinX = velocityMesh.minCoord[0]
        meshDX =   velocityMesh.maxCoord[0] - velocityMesh.minCoord[0]

        for row in range(0,meshY+1):
            vpoints[row,:,0] = new_xVcoords[:] * meshDX + meshMinX

        # This will have changed the mesh since {v,p}points is a view into the original mesh data    

        actual_column = 0
        actual_coord = 0.0
        distance = 1.0

        for i in range(0,meshX+1):
            this_distance = np.abs(new_xVcoords[i] - alpha)
            if this_distance < distance:
                distance = this_distance
                actual_coord = new_xVcoords[i]
                actual_column = i

        return actual_coord, actual_column


In [ ]:
# a function to calculate the response for a depth / wavenumber combo

def single_layer_temperature(wavenumber, layer_height):
    """
    Place a harmonic 'delta function' at a given layer in the mesh.   
    Assumes global: temperatureField, velocityMesh, meshX, meshY
    """

    import numpy as numpy

    
    k = wavenumber * math.pi

    T = temperatureField.data.reshape(meshY+1,meshX+1)
    X = velocityMesh.data.reshape(meshY+1,meshX+1,2)

    delta_width = X[layer_height+1,0,1] - X[layer_height-1,0,1]

    T[...] = 0.0
    T[layer_height,:] = 2.0 * numpy.cos( k * X[layer_height,:,0] ) / delta_width
    

    return T.reshape((meshX+1)*(meshY+1),1)

def layer_fft(data):
    """
    Find fft of data on a mesh layer - reflect data to 
    explicitly impose bc's (see my DPhil code !)
    """

    from numpy import fft 
    import numpy as np
    
    samples = len(data)
    packed_data = np.empty(2*samples-1)
    packed_data[0:samples] = data[:]
    packed_data[samples:2*samples] = data[samples:0:-1]

    dataFT = np.real(fft.rfft(packed_data[:-1])) / (samples-1.0)
    
    return dataFT

# Needs to be modified for heights defined by numpy.linspace, not every level in the mesh.

def direct_wavenumber_topo_kernel(wavenumber, samples):
    """
    Compute the response at a given wavenumber for each layer at depth and return kernel for the same wavenumber
    Assumes global: temperatureField, velocityMesh, meshX, meshY, 
    """

    import numpy as np
        
    height = np.linspace(0.0, 1.0, samples)
    kernel = np.empty(samples)    
    kernel[0] = 0
    kernel[-1] = 1.0
    
    for level in range(1,samples-1):
        
        actual_y, actual_layer  = remap_vmesh_pmesh_y(height[level],0.1, 2.0)     
        temperatureField.data[:] = single_layer_temperature(wavenumber, actual_layer) 
        
        stokesEqn.solve()   
        surface_topography = -verticalStressFn.evaluate(velocityMesh.specialSets["MaxJ_VertexSet"]).T[0]

        kernel[level] = layer_fft(surface_topography)[wavenumber]
        height[level] = actual_y
                
    return kernel, height
 
    
def all_wavenumber_topo_kernel(wavenumber, samples):
    """
    Compute the response at a given wavenumber for each layer at depth and return kernel for the same wavenumber
    Assumes global: temperatureField, velocityMesh, meshX, meshY, 
    """

    import numpy as np

    height = np.linspace(0.0, 1.0, samples)
    kernels = np.empty((samples,meshX+1))
    kernels[0][:] = 0
    kernels[-1][:] = 0
    kernels[-1][wavenumber] = 1.0
    
    for level in range(1,samples-1):

        actual_y, actual_layer  = remap_vmesh_pmesh_y(height[level],0.1, 2.0)     
        temperatureField.data[:] = single_layer_temperature(wavenumber, actual_layer) 
         
        stokesEqn.solve()   
        surface_topography = -verticalStressFn.evaluate(velocityMesh.specialSets["MaxJ_VertexSet"]).T[0]

        kernels[:][level] = layer_fft(surface_topography)
        height[level] = actual_y
            
    return kernels, height

 

### Mesh refinement visualised

Set the density function and see how well the generated mesh-node distribution matches. Note that there is a degree of smoothing in the algorithm which will tend to cut corners on sharp density variations (I see this as a positive right now !)

In [ ]:
# 1) some random periodic variation
    
yd = np.linspace(0.0,1.0,1000)   

separation = 1.0 * np.sin(4.0*math.pi*yd)**2
ys, sep = unit_interval_node_spacer(65, separation, yd)
   
    
fig, (plot1) = pyplot.subplots(1,1)
fig.set_size_inches(16,4)

plot1.plot(ys, sep, marker='x')
plot1.plot(yd, separation, marker='.')
plot1.plot(ys, 0.0*ys, marker='o')


plot1.set_xlim(0,1)

# 2) 3 layer thingo 

separation = 2.0 - 2.0 * three_way_node_density(0.10, 0.25, yd)

ys, sep = unit_interval_node_spacer(65, separation, yd)


fig, (plot1) = pyplot.subplots(1,1)
fig.set_size_inches(16,4)

plot1.plot(ys, sep, marker='x')
plot1.plot(yd, separation, marker='.')
plot1.plot(ys, 0.0*ys, marker='o')


plot1.set_xlim(0,1)

In [ ]:
actual_y, layer  = remap_vmesh_pmesh_y(0.05,0.1, 1.0) 
#actual_x, column = remap_vmesh_pmesh_x(0.4,0.05,1.0) 

y_coordinates = yCoordFn.evaluate(velocityMesh.specialSets["MinI_VertexSet"])

print y_coordinates[[layer-1, layer, layer+1]]
print actual_y, layer

meshfig = plt.Figure()
meshfig.Mesh(velocityMesh, colourBar=False)
meshfig.show()

In [ ]:
# Test this 

level = 7

temperatureField.data[:] = single_layer_temperature(2.0, level)
solver.solve()
vscale = velocityField.evaluate(velocityMesh).max()

figtemp = plt.Figure(figsize=(512,512))
figtemp.VectorArrows(velocityField, velocityMesh, lengthScale=0.1/vscale, resolutionX=41, resolutionY=41)
figtemp.Surface(temperatureField, velocityMesh, colours=["red", "white", "blue"], colourBar=True, logScale=False, opacity=0.3)
figtemp.show()

surface_topography = -verticalStressFn.evaluate(velocityMesh.specialSets["MaxJ_VertexSet"]).T[0]
x_coordinates = xCoordFn.evaluate(velocityMesh.specialSets["MaxJ_VertexSet"])
y_coordinates = yCoordFn.evaluate(velocityMesh.specialSets["MinI_VertexSet"])

import matplotlib.pyplot as pyplot

fig, (plot1,plot2) = pyplot.subplots(1,2)
fig.set_alpha=0.5
fig.set_size_inches(8,4)

plot1.plot(x_coordinates, surface_topography)

surface_topography_fft = layer_fft(surface_topography)

plot2.bar(range(0,len(surface_topography_fft)), surface_topography_fft*100, color="red", alpha=0.3, width=0.8)
plot2.bar(range(0,len(surface_topography_fft)), surface_topography_fft)

plot2.set_xlim(0,20)
plot2.set_ylim(-0.05,1.05)

pass

# Seems we get the right answer ... 

vpoints = velocityMesh.data.reshape(meshY+1,meshX+1,2)
print "Evaluated at depth range - ", vpoints[level-1,0,1], vpoints[level,0,1], vpoints[level+1,0,1]
print "Kernel value - ", surface_topography_fft[2]


In [ ]:
figtemp = plt.Figure(figsize=(512,512))
figtemp.VectorArrows(velocityField, velocityMesh, lengthScale=0.1/vscale, resolutionX=41, resolutionY=41)
#figtemp.Mesh(velocityMesh)
figtemp.Surface(temperatureField, velocityMesh, colours=["red", "white", "blue"], colourBar=True, logScale=False, opacity=0.3)
figtemp.show()

In [ ]:
K, Y = direct_wavenumber_topo_kernel(2.0, 20)

In [ ]:
# read uniform mesh example and compare:

uniform_case = np.load("Data/IsoviscousKernel2.npz")
uy = uniform_case['y']
uk = uniform_case['kernel']

fig, (plot1) = pyplot.subplots(1,1)
fig.set_size_inches(12,4)
plot1.plot(K,  Y, marker='o')
plot1.plot(uk, uy, marker='x')


In [ ]:
# Now the same comparison but with a layered viscosity

# Change the viscosity values which were defined as mutable constant functions

layer1Viscosity.value = 100.0
layer2Viscosity.value = 0.1
layer3Viscosity.value = 1.0


In [ ]:
K, Y = all_wavenumber_topo_kernel(2.0, 33)

In [ ]:
# read uniform mesh example and compare:

uniform_case = np.load("Data/ThreeLayer_100_01_1_Kernel2.npz")
uy = uniform_case['y']
uk = uniform_case['kernel']

fig, (plot1) = pyplot.subplots(1,1)
fig.set_size_inches(12,4)
plot1.plot(K[:,2],  Y, marker='o')
plot1.plot(K[:,3],  Y, marker='.')
plot1.plot(uk, uy, marker='x')

# Note the result is not that bad considering that we don't try to resolve the 
# horizontal layer location with the refinement function. 